In [2]:
import os
import sys

import requests

import numpy as np

from astropy import units as u
from astropy import table
from astropy.coordinates import SkyCoord, EarthLocation, get_sun
from astropy.time import Time
from astropy.io import fits

from importlib import reload

%matplotlib inline
from matplotlib import style, pyplot as plt

Most of the initial steps are in 2018Oct-AAT

# Initial imports/initialization of SAGA objects 

In [3]:
working_dir = os.path.abspath('..')

%cd ../../saga
sagadir = !pwd
sagadir = sagadir[0]
!python setup.py build
%cd $working_dir

/Users/erik/projects/saga/saga
running build
running build_py
/Users/erik/projects/saga/erik


In [4]:
sagalibdir = os.path.join(sagadir, 'build/lib')
print('Adding', sagalibdir, 'to the python path')
sys.path.insert(0, sagalibdir)

import SAGA as saga
saga.__version__

Adding /Users/erik/projects/saga/saga/build/lib to the python path


'0.10.10'

In [17]:
saga_database = saga.Database(os.path.join(os.environ['HOME'],'Dropbox/SAGA/'))

saga_host_catalog = saga.HostCatalog(saga_database)
saga_object_catalog = saga.ObjectCatalog(saga_database)
saga_targets = saga.TargetSelection(saga_database, manual_selected_objids='manual_targets_aat2018b')#,
                                    #assign_targeting_score_kwargs=dict(ignore_specs=True))

In [18]:
hosts_to_load = 'aat_2018b'
host_id_label = 'ID_STR'
targethosts = saga_host_catalog.load(hosts_to_load)
targethosts[host_id_label] = saga_host_catalog.resolve_id(hosts_to_load, 'string')


# used below to save generated catalogs
acats = {}
tcats = {}

# Helper functions (to eventually be moved to saga repo)

In [7]:
def get_gaia_guidestars(hostname, magrng=(12.5, 13.5),
                        matchmagrng=(16, 17), d_matchmag=1, matchtol0=1*u.arcsec,
                        verbose=True, neighbor_cut=30*u.arcsec, nmagdown=4.5):
    """
    `magrng` is the range of magnitudes to actually select on.  It's ~r-band, based on the
    Evans et al. 2018 r-to-G conversion
    """
    print = __builtin__.print
    if not verbose:
        print = lambda *args, **kwargs:None
        
    host = saga_host_catalog.load_single(hostname)
    obj_cat = saga_object_catalog.load(hostname)[0]
    omag = obj_cat['r_mag']
    gaia_cat = table.Table.read('external_catalogs/astrometric/{}_gaia.ecsv'.format(hostname))
    gaia_sc = SkyCoord(gaia_cat['ra'], gaia_cat['dec'])
    gmag = gaia_cat['phot_g_mean_mag']
    
    omsk = (matchmagrng[0]<omag)&(omag<matchmagrng[1])
    gmsk = ((matchmagrng[0]-d_matchmag)<gmag)&(gmag<(matchmagrng[1]+d_matchmag))
    oscmsk = obj_cat['coord'][omsk]
    gscmsk = gaia_sc[gmsk]
    
    idx, d2d, _ = oscmsk.match_to_catalog_sky(gscmsk)
    sepmsk = d2d < matchtol0
    
    dra = (oscmsk[sepmsk].ra - gscmsk[idx][sepmsk].ra).to(u.arcsec)
    ddec = (oscmsk[sepmsk].dec - gscmsk[idx][sepmsk].dec).to(u.arcsec)
    offset = np.mean(dra), np.mean(ddec)
    
    print("Object catalog to Gaia offset:", offset, 'from', np.sum(sepmsk), 
          'objects ({:.1%})'.format(np.sum(sepmsk)/len(sepmsk)))
    
    #this polynomial is from Evans et al. 2018 for the G to r conversion
    Gmr_coeffs = (-0.1856, 0.1579, 0.02738, -0.0550)
    Gmr = np.polyval(Gmr_coeffs[::-1], gaia_cat['bp_rp'])
    gaia_cat['g_as_r_mag'] = gmag - Gmr
    
    gmsk = (magrng[0] < gaia_cat['g_as_r_mag'])&(gaia_cat['g_as_r_mag'] < magrng[1])
    gstars = gaia_cat[gmsk]
    print('Found', len(gstars), 'Gaia guide stars')
    
    if neighbor_cut is not None:
        possible_neighbor_stars = gaia_cat[gaia_cat['g_as_r_mag'] < magrng[1]+nmagdown]
        nsc = SkyCoord(possible_neighbor_stars['ra'], possible_neighbor_stars['dec'])
        gsc = SkyCoord(gstars['ra'], gstars['dec'])
        idx, d2d, _ = gsc.match_to_catalog_sky(nsc, 2)
        neighbor_present = d2d < neighbor_cut
        print(np.sum(neighbor_present), 'Have a brightish neighbor.  Removing them.')
        gstars = gstars[~neighbor_present]

    tab = table.Table({
        'TargetName': gstars['source_id'],
        'RA': (u.Quantity(gstars['ra']) + offset[0]),
        'Dec': (u.Quantity(gstars['dec']) + offset[1]),
        'TargetType': np.repeat('F', len(gstars)),
        'Priority': np.repeat(9, len(gstars)),
        'Magnitude':gstars['g_as_r_mag'],
        '0': np.repeat(0, len(gstars)),
        'Notes': np.repeat('guide_gaia', len(gstars)),
    })
    tab.meta['dra'] = dra
    tab.meta['ddec'] = ddec
    tab.meta['offset'] = offset
    tab.meta['gstarcat'] = gstars
    return tab

In [8]:
def get_sdss_guidestars(hostname):
    host = saga_host_catalog.load_single(hostname)
    obj_cat = saga_object_catalog.load(hostname)[0]
    r = obj_cat['r_mag']
    msk = (12.5<r)&(r<14)&~obj_cat['is_galaxy']&(obj_cat['RHOST_ARCM']>15)
    starcat = obj_cat[msk]
    
    print('Found', len(starcat), 'SDSS guide stars')
    
    return table.Table({
        'TargetName': starcat['OBJID'],
        'RA': starcat['RA'],
        'Dec': starcat['DEC'],
        'TargetType': np.repeat('F', len(starcat)),
        'Priority': np.repeat(9, len(starcat)),
        'Magnitude': starcat['r_mag'],
        '0': np.repeat(0, len(starcat)),
        'Notes': np.repeat('guide', len(starcat)),
    })

In [204]:
from astropy.coordinates import Angle

def write_fld_file(target_catalog, host, obstime, fn, suffix=' master catalog'):
    hostname = host[host_id_label]
    target_catalog.write(fn,
                         delimiter=' ',
                         quotechar='"',
                         format='ascii.fast_commented_header',
                         overwrite=True,
                         formats={
                            'RA': lambda x: Angle(x, 'deg').wrap_at(360*u.deg).to_string('hr', sep=' ', precision=2), # pylint: disable=E1101
                            'Dec': lambda x: Angle(x, 'deg').to_string('deg', sep=' ', precision=2),
                            'Magnitude': '%.2f',
                         })
    with open(fn) as fh:
        content = fh.read()
    with open(fn, 'w') as fh:
        fh.write('LABEL ' + hostname + suffix + '\n')
        fh.write('UTDATE  {yr} {mo:02} {day:02}\n'.format(yr=obstime.datetime.year,
                                                        mo=obstime.datetime.month,
                                                        day=obstime.datetime.day))
        censtr = host['coord'].to_string('hmsdms', sep=' ', precision=2, alwayssign=True)
        fh.write('CENTRE  ' + censtr + '\n')
        fh.write('EQUINOX J2000.0\n')
        fh.write('# End of Header\n\n')
        fh.write(content.replace('"', ''))
        

def subsample_catalog(catalog, prilimits=None, maxflux=np.inf, maxguides=np.inf, maxsky=np.inf, verbose=True, favor_highpri=False):
    """
    Subsamples the catalog, limiting the number in a given priority set
    
    ``prilimits`` should be a dictionary mapping priority number to the maximum number of
    objects in that priority.  If it is None, no limits will be used.
    
    ``favor_highpri`` determines if the prilimits cuts pick favored (it must be an array 
    which matches ``catalog``, where  *lower* numbers are favored), ora random sampling (False)
    """
    
    
    msks = [catalog['Notes']==notestr for notestr in ('Flux', 'Sky')]
    msks.append(np.array([n.lower().startswith('guide') for n in catalog['Notes']]))
    maxns = [maxflux, maxsky, maxguides]
    
    
    
    if prilimits is not None:
        for pri, maxn in prilimits.items():
            msks.append(catalog['Priority']==pri)
            maxns.append(maxn)
            
    idxs_to_rem = []
    for i, (msk, maxn) in enumerate(zip(msks, maxns)):
        ntorem = np.sum(msk) - maxn
        if ntorem >= 1:
            idxs = np.where(msk)[0]
            if favor_highpri is False or i < 3:
                idxs_to_rem.append(np.random.permutation(idxs)[:ntorem])
            else:
                assert len(favor_highpri) == len(catalog), 'catalog and favor_highpri do not match in length!'
                idxs_to_rem.append(idxs[np.argsort(favor_highpri[idxs])][-ntorem:])
    subcat = catalog.copy()
    if idxs_to_rem:
        del subcat[np.concatenate(idxs_to_rem)]
    
    if verbose:
        npris = dict(enumerate(np.bincount(subcat['Priority'][subcat['Notes']=='Targets'])))
        npris = {k:v for k,v in npris.items() if v>0}
        print('Nflux:', np.sum(subcat['Notes']=='Flux'), 
              'Nguide:', np.sum([n.lower().startswith('guide') for n in subcat['Notes']]), 
              'NSky', np.sum(subcat['Notes']=='Sky'))
        print('Targets in each priority:', npris)
        print('Total:', len(subcat))
        
    return subcat

In [10]:
def infer_radec_cols(table):
    raname = decname = None
    for cnm in table.colnames:
        if cnm.lower()=='ra':
            raname = cnm
        elif cnm.lower()=='dec':
            decname = cnm
    return raname, decname

In [11]:
def make_decals_viewer_cutouts(table, survey='sdss', ncols=3, zoom=15, size=120, namecol=None, dhtml=True):
    """
    Zoom of 15 is ~1"/pixel, so ~2' across with defaults
    """
    template_url = 'http://legacysurvey.org/viewer/jpeg-cutout/?ra={ra:.7}&dec={dec:.7}&zoom={zoom}&layer={layer}&size={size}'
    
    raname, decname = infer_radec_cols(table)
    
    entries = []
    for row in table:
        imgurl = template_url.format(ra=row[raname], dec=row[decname], layer=survey, size=size, zoom=zoom)
        viewurl = 'http://legacysurvey.org/viewer?ra={}&dec={}'.format(row[raname], row[decname])
        
        namestr = '' if namecol is None else (str(row[namecol]) + '<br>')
        entries.append('{}<a href="{}"><img src="{}"></a>'.format(namestr, viewurl, imgurl))
        
    entryrows = [[]]
    while entries:
        entry = entries.pop(0)
        if len(entryrows[-1]) >= ncols:
            entryrows.append([])
        entryrows[-1].append(entry)
    entryrows[-1].extend(['']*(ncols-len(entryrows[-1])))
    
    tabrows = ['<td>{}</td>'.format('</td><td>'.join(erow)) for erow in entryrows]

    htmlstr = """
    <table>
    <tr>{}</tr>
    </table>
    """.format('</tr>\n<tr>'.join(tabrows))

    if dhtml:
        from IPython import display
        return display.HTML(htmlstr)
    else:
        return htmlstr

In [12]:
def make_des_cutouts_file(table, copytoclipboard=True, showtable=False):
    raname, decname = infer_radec_cols(table)
    
    entries = []
    for row in table:
        entries.append('{:.5f},{:.5f}'.format(row[raname], row[decname]))
        
    htmlstr = """
    Paste the coords in "Enter values" at  
    <a href="https://des.ncsa.illinois.edu/easyweb/cutouts">the DES cutout service</a>,
    and change the size to 0.3 arcmin
    """
    tablestr = """<br>
    <table>
    <tr><td>{}</td></tr>
    </table>
    """.format('</td></tr>\n<tr><td>'.join(entries))
    if showtable:
        htmlstr += tablestr
    
    if copytoclipboard:
        import platform
        text = '\n'.join(entries)
        if platform.system() == 'Darwin':
            clipproc = os.popen('pbcopy', 'w')
            clipproc.write(text)
            clipproc.close()
            print('Copied to clipboard')
        elif platform.system() == 'Linux':
            clipproc = os.popen('xsel -i', 'w')
            clipproc.write(text)
            clipproc.close()
            print('Copied to clipboard')
        else:
            raise OSError("Not on a mac or linux, so can't use clipboard. ")
        
    from IPython import display
    return display.HTML(htmlstr)
    
def show_des_cutouts(table, jobname, username='eteq', ncols=3, namecol=None, force_size=(128, 128), dhtml=True):
    base_url = 'https://des.ncsa.illinois.edu'
    
    list_url = base_url + '/easyweb/static/workdir/{}/{}/list.json'.format(username, jobname)
    print(list_url)
    list_json = requests.get(list_url).json()
    img_urls = [base_url+img['name'] for img in list_json]
    
    if len(table) != len(img_urls):
        raise ValueError('table and job length do not match!')
    raname, decname = infer_radec_cols(table)
    
    sizestr = '' if force_size is None else ('height="{}" width="{}"'.format(force_size[1], force_size[0]))
    entries = []
    for row, imgurl in zip(table, img_urls):
        viewurl = 'http://legacysurvey.org/viewer?ra={}&dec={}'.format(row[raname], row[decname])
        
        namestr = '' if namecol is None else (str(row[namecol]) + '<br>')
        entries.append('{}<a href="{}"><img src="{}"{}></a>'.format(namestr, viewurl, imgurl, sizestr))
        
    entryrows = [[]]
    while entries:
        entry = entries.pop(0)
        if len(entryrows[-1]) >= ncols:
            entryrows.append([])
        entryrows[-1].append(entry)
    entryrows[-1].extend(['']*(ncols-len(entryrows[-1])))
    
    tabrows = ['<td>{}</td>'.format('</td><td>'.join(erow)) for erow in entryrows]

    htmlstr = """
    <table>
    <tr>{}</tr>
    </table>
    """.format('</tr>\n<tr>'.join(tabrows))

    if dhtml:
        from IPython import display
        return display.HTML(htmlstr)
    else:
        return htmlstr


# Configuration File-Generation Templates 

## nsaxxxx

In [ ]:
idnum = xxxx
fieldnum = NN

saga_object_catalog.build_and_write_to_database('NSA{}'.format(idnum), overwrite=fieldnum>1)

In [ ]:
host = targethosts[targethosts['NSAID'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_sdss_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

In [ ]:
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))], 
                           survey='sdss',ncols=6,namecol='TargetName')

In [ ]:
guides_to_rem = [int(i) for i in 'nnnn nnn nnnn'.split()]
writecat.remove_rows((writecat['Notes']=='Guide')&(np.in1d(writecat['TargetName'], guides_to_rem)))
np.sum((writecat['Notes']=='Guide'))

In [ ]:
make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='sdss', ncols=6, 
                           zoom=16, namecol='TargetName')

In [ ]:
skys_to_rem = [int(i) for i in 'n n nn nn'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

In [ ]:
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

In [ ]:
!scp $fldfn visitor8@aatlxh:~/octDD/

## pgcxxxx

In [ ]:
idnum = xxxx
fieldnum = NN

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

In [ ]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

In [ ]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

In [ ]:
#The below is only necessary if the decals viewer is broken but the targets are in DES
# make_des_cutouts_file(acat[acat['Notes']=='sky'])
# show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')

make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='des-dr1', namecol='TargetName',
                           ncols=8, zoom=15, size=120)

In [ ]:
skys_to_rem = [int(i) for i in <'n n nn'>.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

In [ ]:
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

In [ ]:
!scp $fldfn visitor8@aatlxh:~/octDD/

# Night 2

In [39]:
import aat

obs_start_date = Time('2018-8-4') 

## pgc67663

In [31]:
idnum = 67663
fieldnum = 2  # 1 was done in a previous run

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

[10/04 19:43:48] [WARNING] Not found: SDSS catalog for pgc67663.
[10/04 19:43:48] [WARNING] Not found: DECALS catalog for pgc67663.
[10/04 19:43:48] Use DES to build base catalog v2 for pgc67663 (1/1)


[10/04 19:44:12] Write base catalog to ./base_catalogs/base_v2_pgc67663.fits.gz


In [32]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

pgc67663
21h56m09.6s -49d31m19.2s
# of flux stars = 207
# of sky fibers = 100
# of Priority=1 targets = 785
# of Priority=2 targets = 130
# of Priority=3 targets = 29
# of Priority=4 targets = 50
# of Priority=5 targets = 158
# of Priority=6 targets = 102
# of Priority=7 targets = 109
# of Priority=8 targets = 11
# of Priority=9 targets = 307
Object catalog to Gaia offset: (<Angle -0.12399641 arcsec>, <Angle 0.14305041 arcsec>) from 1117 objects (22.2%)
Found 187 Gaia guide stars
63 Have a brightish neighbor.  Removing them.


1805

In [35]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

6559423764127892480,6559190117907060992,6559332916979834112,6559259833816232448,6559406137582078592,6559178813553069440
6560902121934858752,6559081506773933312,6559066251050018944,6559195615465216256,6559276807526995200,6560619925403096960
6559354460535531264,6559402151852497152,6559166409687563520,6559427440619902976,6560690362866747776,6559099232102507648
6559220972952048000,6559071336291363584,6558997802155435264,6558981519935088000,6559165275816192640,6560713594346909568


In [36]:
#The below is only necessary if the decals viewer is broken but the targets are in DES
# make_des_cutouts_file(acat[acat['Notes']=='sky'])
# show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')

make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='des-dr1', namecol='TargetName',
                           ncols=8, zoom=15, size=120)

0,1,2,3,4,5,6,7
8,9,10,11,12,13,14,15
16,17,18,19,20,21,22,23
24,25,26,27,28,29,30,31
32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47
48,49,50,51,52,53,54,55
56,57,58,59,60,61,62,63
64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87


In [37]:
skys_to_rem = [int(i) for i in '2 23 27 47 50 51 61 62 66 73 75 83'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

88

In [44]:
subsampledcat = subsample_catalog(writecat, {1:200}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

Nflux: 5 Nguide: 100 NSky 88
Targets in each priority: {1: 200, 2: 130, 3: 29, 4: 50, 5: 158, 6: 102, 7: 109, 8: 11}
Total: 982
Writing aat_targets_oct2018/pgc67663_2.fld


789

In [45]:
!scp $fldfn visitor8@aatlxh:~/oct04/

pgc67663_2.fld                                100%   53KB  53.2KB/s   00:00    


## pgc1952

In [82]:
idnum = 1952
fieldnum = 1

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

[10/04 20:11:47] Base catalog v2 for pgc1952 already exists (./base_catalogs/base_v2_pgc1952.fits.gz). (1/1)


In [83]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

pgc1952
00h32m02.112s -64d15m12.24s
# of flux stars = 141
# of sky fibers = 100
# of Priority=1 targets = 490
# of Priority=2 targets = 109
# of Priority=3 targets = 10
# of Priority=4 targets = 50
# of Priority=5 targets = 115
# of Priority=6 targets = 62
# of Priority=7 targets = 42
# of Priority=8 targets = 5
# of Priority=9 targets = 241
Object catalog to Gaia offset: (<Angle -0.36644631 arcsec>, <Angle 0.07111843 arcsec>) from 921 objects (72.6%)
Found 171 Gaia guide stars
46 Have a brightish neighbor.  Removing them.


1249

In [84]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

4708349892420964224,4708457030381329792,4708380064567371776,4708184866893917824,4900805560184448384,4900604074678797568
4900541471235457664,4708274064774704000,4900660355930222464,4708480257564404480,4900613454887348864,4708503278589078400
4708286399920774400,4900642660665060736,4708219295351729408,4900615275953476608,4708226648335664512,4708436586336930432
4900636304113437312,4708478677016445184,4709732841826766976,4708367385824049024,4900803532959886464,4900633417895436800


In [85]:
#The below is only necessary if the decals viewer is broken but the targets are in DES
# make_des_cutouts_file(acat[acat['Notes']=='sky'])
# show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')

make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='des-dr1', namecol='TargetName',
                           ncols=8, zoom=15, size=120)

0,1,2,3,4,5,6,7
8,9,10,11,12,13,14,15
16,17,18,19,20,21,22,23
24,25,26,27,28,29,30,31
32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47
48,49,50,51,52,53,54,55
56,57,58,59,60,61,62,63
64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87


In [86]:
skys_to_rem = [int(i) for i in '1 5 9 11 15 16 18 20 22 24 27 29 35 41 43 50 54 55 58 59 60 62 65 67 69 71 75 78 77 79 90 91 93 96 97 98'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

64

In [90]:
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

Nflux: 5 Nguide: 100 NSky 64
Targets in each priority: {1: 490, 2: 109, 3: 10, 4: 50, 5: 115, 6: 62, 7: 42, 8: 5}
Total: 1052
Writing aat_targets_oct2018/pgc1952_1.fld


883

In [91]:
!scp $fldfn visitor8@aatlxh:~/oct04/

pgc1952_1.fld                                 100%   56KB  56.2KB/s   00:00    


## pgc10965

In [103]:
idnum = 10965
fieldnum = 1

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

[10/05 00:01:27] Base catalog v2 for pgc10965 already exists (./base_catalogs/base_v2_pgc10965.fits.gz). (1/1)


In [104]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

pgc10965
02h54m33.48s -18d38m06s


/Users/erik/miniconda3/envs/saga3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


# of flux stars = 53
# of sky fibers = 100
# of Priority=1 targets = 667
# of Priority=2 targets = 76
# of Priority=3 targets = 22
# of Priority=4 targets = 50
# of Priority=5 targets = 126
# of Priority=6 targets = 92
# of Priority=7 targets = 39
# of Priority=8 targets = 4
# of Priority=9 targets = 153


/Users/erik/miniconda3/envs/saga3/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)
/Users/erik/miniconda3/envs/saga3/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


Object catalog to Gaia offset: (<Angle -0.1295607 arcsec>, <Angle 0.0761455 arcsec>) from 592 objects (70.5%)
Found 118 Gaia guide stars
26 Have a brightish neighbor.  Removing them.


1321

In [105]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

5153246862915935488,5128257647971759616,5128410132195052288,5152452951096870784,5128460507867291136,5128237302711063936
5152411508956735616,5153285762435428096,5152504314610058880,5152501879364274304,5129197863557045248,5128361586180577920
5128372718735802112,5152524281913703168,5128402298175011584,5129222362050343552,5128332346042971264,5152475078768369152
5128176799507528960,5128130276421787264,5152408210421852672,5129209275285512448,5128188481818570880,5129245181212092160


In [106]:
#The below is only necessary if the decals viewer is broken but the targets are in DES
# make_des_cutouts_file(acat[acat['Notes']=='sky'])
# show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')

make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='des-dr1', namecol='TargetName',
                           ncols=8, zoom=15, size=120)

0,1,2,3,4,5,6,7
8,9,10,11,12,13,14,15
16,17,18,19,20,21,22,23
24,25,26,27,28,29,30,31
32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47
48,49,50,51,52,53,54,55
56,57,58,59,60,61,62,63
64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87


In [107]:
skys_to_rem = [int(i) for i in '10 12 19 23 25 27 28 35 36 45 50 53 55 62 67 75 77 80 89 90 91 96'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

78

In [108]:
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

Nflux: 5 Nguide: 92 NSky 78
Targets in each priority: {1: 667, 2: 76, 3: 22, 4: 50, 5: 126, 6: 92, 7: 39, 8: 4}
Total: 1251
Writing aat_targets_oct2018/pgc10965_1.fld


1076

In [109]:
!scp $fldfn visitor8@aatlxh:~/oct04/

pgc10965_1.fld                                100%   66KB  66.5KB/s   00:00    


# Scratch/working space 

## Check fibers from list file (mostly for guidestar when on sky) 

In [ ]:
lt, sc, _ = aat.load_lis_file('aat_targets_oct2018/nsa13927_1_p0.lis')
lt['ra'] = sc.ra.deg
lt['dec'] = sc.dec.deg

In [ ]:
make_decals_viewer_cutouts(lt[lt['codes']=='F'], 'sdss', namecol='fibnums', ncols=4)

In [ ]:
make_decals_viewer_cutouts(lt[lt['codes']=='F'], 'unwise-w1w2', namecol='mags', ncols=4, zoom=17)

In [101]:
import tqdm

testcats = {}
for hostname, cat in saga_targets.build_target_catalogs(hosts_to_load, return_as='items', reload_base=True):
    testcats[hostname] = cat
    

# MAKE build_target_catalogs EASIER TO USE: accept lists of hosts. 

/Users/erik/miniconda3/envs/saga3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


In [102]:
base_bins = np.array([1, 100, 150, 180, 200, 300, 400, 500, 600, 700, 800, 900, 1000])
score_bins  = base_bins+1
cum_bins = base_bins-.1

count_tab = table.Table()
count_tab.add_column(table.Column(name='name', dtype='U10'))
cum_tab = table.Table()
cum_tab.add_column(table.Column(name='name', dtype='U10'))
for b1, b2 in zip(score_bins[:-1], score_bins[1:]):
    count_tab.add_column(table.Column(name='bin_{}to{}'.format(b1, b2), dtype=int))
for b in base_bins[2:]:
    cum_tab.add_column(table.Column(name='lt_{}'.format(b), dtype=int))

#for nm, cat in testcats.items():
for nm, cat in testcats.items():
    nbins = np.histogram(cat['TARGETING_SCORE'], score_bins)[0]
    count_tab.add_row([nm] + list(nbins))
    nbins = np.histogram(cat['TARGETING_SCORE'], cum_bins)[0][1:]
    cum_tab.add_row([nm] + list(np.cumsum(nbins)))
    
count_tab#.pprint(max_width=1000)

name,bin_2to101,bin_101to151,bin_151to181,bin_181to201,bin_201to301,bin_301to401,bin_401to501,bin_501to601,bin_601to701,bin_701to801,bin_801to901,bin_901to1001
str10,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
pgc1952,5,1,0,0,48,63,118,50,10,128,503,3400
pgc2052,8,0,22,0,29,50,67,48,9,23,756,2185
pgc9747,8,1,0,0,34,62,182,50,6,44,783,2962
pgc10965,4,0,0,0,39,92,126,50,22,76,667,2611
nsa133355,6,1,0,0,61,90,124,50,30,96,620,2888
pgc64427,2,0,7,0,28,5,7,48,36,182,128,4136
pgc66934,2,1,1,0,12,11,22,50,28,159,224,3516
pgc67146,2,0,0,0,29,27,52,50,17,102,459,3805
pgc67663,11,0,0,0,109,102,158,50,29,130,785,2614


In [30]:
cum_tab

name,lt_150,lt_180,lt_200,lt_300,lt_400,lt_500,lt_600,lt_700,lt_800,lt_900,lt_1000
str10,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
pgc1952,5,5,5,47,109,224,274,284,393,883,3262
pgc2052,8,8,29,32,79,136,185,194,204,934,2104
pgc9747,8,9,9,35,96,275,325,330,366,1135,3101
pgc10965,4,4,4,32,123,246,296,315,379,1022,2732
nsa133355,6,6,6,57,145,267,317,345,429,1046,2995
pgc64427,2,2,8,9,12,15,64,95,170,262,3194
pgc66934,2,2,3,3,13,29,79,105,220,432,2944
pgc67146,2,2,2,8,34,83,133,150,225,673,3385
pgc67663,11,11,11,120,222,380,430,459,589,1374,3173


In [48]:
specft = table.Table.read('SAGADropbox/Spectra/Final/AAT/pgc67663_1_MG.fits.gz')
specft

NAME,RA,DEC,X,Y,XERR,YERR,THETA,TYPE,PIVOT,MAGNITUDE,PID,COMMENT,RETRACTOR,WLEN,PMRA,PMDEC
bytes80,float64,float64,int32,int32,int16,int16,float64,bytes1,int16,float64,int32,bytes80,bytes10,float64,float64,float64
244138713,5.753572417867123,-0.8645098174740619,-137,93473,12,-2,0.014868,P,1,20.54,0,Targets,68,600.0,0.0,0.0
244139545,5.758745137437722,-0.864807008260582,-3263,139284,14,-1,0.027267,P,2,20.61,0,Targets,68,600.0,0.0,0.0
244136898,5.747928459398487,-0.8640170528485821,5291,44049,16,-5,0.070104,P,3,16.77,0,Targets,68,600.0,0.0,0.0
244660511,5.761096968604784,-0.8652263720947417,-8583,160465,17,-7,0.057833,P,4,17.42,0,Flux,68,600.0,0.0,0.0
244139768,5.752873558945804,-0.864898783490416,-5469,87450,2,14,0.074141,P,5,20.55,0,Targets,68,600.0,0.0,0.0
244138851,5.751611831340717,-0.8645481662362376,-1014,76318,-4,8,0.112673,P,6,20.17,0,Targets,68,600.0,0.0,0.0
FIBRE NOT IN USE,5.744975163833825,-0.8432789911712496,-27715,269846,0,0,0.0785,N,7,0.0,0,Parked,68,0.0,0.0,0.0
244660347,5.75471997185031,-0.8651456021354689,-8458,103745,18,-5,0.135644,P,8,20.23,0,Targets,68,600.0,0.0,0.0
244139800,5.750455550711271,-0.8649180790749241,-6200,66352,-1,1,0.142084,P,9,19.91,0,Targets,68,600.0,0.0,0.0


In [67]:
mzt = table.Table.read('SAGADropbox/Spectra/Final/AAT/pgc67663_1_MG.mz', format='ascii.commented_header', header_start=2)
mzt

ID,Name,RA,DEC,Mag,Type,AutoTID,AutoTN,AutoZ,AutoXCor,FinTID,FinTN,FinZ,QOP,Comment
int64,int64,float64,float64,float64,str7,int64,str33,float64,float64,int64,str33,float64,int64,int64
1,244138713,5.753572,-0.86451,20.54,Targets,11,Transitional Galaxy,0.03654,4.97842,11,Transitional Galaxy,0.03654,1,--
2,244139545,5.758745,-0.864807,20.61,Targets,8,Late Type Emission Galaxy,0.27813,10.5824,8,Late Type Emission Galaxy,0.27813,4,--
3,244136898,5.747928,-0.864017,16.77,Targets,8,Late Type Emission Galaxy,0.0655,14.07315,8,Late Type Emission Galaxy,0.0655,4,--
4,244660511,5.761097,-0.865226,17.42,Flux,5,G Star,-6e-05,9.78605,5,G Star,-6e-05,4,--
5,244139768,5.752874,-0.864899,20.55,Targets,8,Late Type Emission Galaxy,0.21662,14.78831,8,Late Type Emission Galaxy,0.21662,4,--
6,244138851,5.751612,-0.864548,20.17,Targets,8,Late Type Emission Galaxy,0.26013,17.03793,8,Late Type Emission Galaxy,0.26013,4,--
8,244660347,5.75472,-0.865146,20.23,Targets,8,Late Type Emission Galaxy,0.28082,10.36827,8,Late Type Emission Galaxy,0.28082,4,--
9,244139800,5.750456,-0.864918,19.91,Targets,8,Late Type Emission Galaxy,0.22056,16.91211,8,Late Type Emission Galaxy,0.22056,4,--
10,245628597,5.757406,-0.865828,17.03,Targets,8,Late Type Emission Galaxy,0.09006,14.69302,8,Late Type Emission Galaxy,0.09006,4,--


In [72]:
specftsub = specft[specft['TYPE']=='P']
np.all(np.in1d(specftsub['NAME'].astype(int), tcat['OBJID']))

True

Objects in the mz file that are being re-observed:

In [76]:
reobserved = mzt[np.in1d(mzt['Name'].astype(int), acat['TargetName'])]
reobserved

ID,Name,RA,DEC,Mag,Type,AutoTID,AutoTN,AutoZ,AutoXCor,FinTID,FinTN,FinZ,QOP,Comment
int64,int64,float64,float64,float64,str7,int64,str33,float64,float64,int64,str33,float64,int64,int64
1,244138713,5.753572,-0.86451,20.54,Targets,11,Transitional Galaxy,0.03654,4.97842,11,Transitional Galaxy,0.03654,1,--
4,244660511,5.761097,-0.865226,17.42,Flux,5,G Star,-6e-05,9.78605,5,G Star,-6e-05,4,--
11,244660548,5.751421,-0.865245,17.53,Flux,5,G Star,-8e-05,9.50522,5,G Star,-8e-05,4,--
15,245632064,5.757417,-0.867141,17.21,Flux,5,G Star,-0.00026,9.89037,5,G Star,-0.00026,4,--
44,245642511,5.755732,-0.871561,19.36,Targets,10,High Redshift Star Forming Galaxy,0.16389,3.64028,10,High Redshift Star Forming Galaxy,0.16389,2,--
45,245647243,5.75783,-0.873531,17.56,Flux,5,G Star,0.00015,9.99133,5,G Star,0.00015,4,--
288,244955394,5.741429,-0.860352,20.65,Targets,10,High Redshift Star Forming Galaxy,0.6421,4.91543,10,High Redshift Star Forming Galaxy,0.6421,1,--
298,244942706,5.742607,-0.855669,17.36,Flux,6,Early Type Absorption Galaxy,-0.00014,10.79983,6,Early Type Absorption Galaxy,-0.00014,4,--
384,244135825,5.748336,-0.863468,19.52,Targets,7,Intermediate Type Galaxy,0.94948,5.16513,7,Intermediate Type Galaxy,0.94948,1,--


Why are thes QOP=4 things getting re-observed?

In [79]:
tcat[np.in1d(tcat['OBJID'], reobserved['Name'])]

OBJID,RA,DEC,REMOVE,is_galaxy,morphology_info,radius,radius_err,u_mag,g_mag,r_mag,i_mag,z_mag,y_mag,u_err,g_err,r_err,i_err,z_err,y_err,survey,OBJID_des,RA_des,DEC_des,REMOVE_des,is_galaxy_des,morphology_info_des,radius_des,radius_err_des,u_mag_des,g_mag_des,r_mag_des,i_mag_des,z_mag_des,y_mag_des,u_err_des,g_err_des,r_err_des,i_err_des,z_err_des,y_err_des,HOST_NSAID,HOST_NSA1ID,HOST_RA,HOST_DEC,HOST_DIST,HOST_VHOST,HOST_MK,HOST_MR,HOST_MG,HOST_SAGA_NAME,HOST_NGC,HOST_PGC,RHOST_ARCM,RHOST_KPC,OBJ_NSAID,SPEC_REPEAT,SPEC_REPEAT_ALL,SPEC_Z,SPEC_Z_ERR,ZQUALITY,SPECOBJID,MASKNAME,TELNAME,EM_ABS,HELIO_CORR,RA_spec,DEC_spec,SATS,sb_r,log_sm,ug,ug_err,gr,gr_err,ri,ri_err,iz,iz_err,zy,zy_err,rz,rz_err,TARGETING_SCORE
int64,float64,float64,int64,bool,int32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str6,int64,float64,float64,int64,bool,int32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,int32,float32,float32,float32,float32,float32,float32,float32,str48,int32,int32,float32,float32,int32,str48,str48,float32,float32,int16,str48,str48,str6,int16,bool,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
245642511,329.779120116802,-49.9367876966731,0,True,3,4.11304343986511,0.0,99.0,20.1798435,19.36431651,19.1507081687408,18.686703118,18.4463,99.0,0.0415101,0.0217748,0.0512396,0.029584,0.0652006,des,245642511,329.779120116802,-49.9367876966731,0,True,3,4.11304343986511,0.0,99.0,20.1798435,19.36431651,19.1507081687408,18.686703118,18.4463,99.0,0.0415101,0.0217748,0.0512396,0.029584,0.0652006,-1,-1,329.04,-49.522,26.8605,1853.0,0.0,0.0,0.0,,7155,67663,37.961666,296.60376,-1,AAT,AAT,0.16388465,3.335641e-05,2,44,pgc67663_1_MG.mz,AAT,-1,True,329.77915160839234,-49.93676688820155,-1,24.430582662545326,nan,78.8201565,99.00000870246629,0.8155269900000022,0.04687462338035368,0.21360834125919936,0.05567439737617283,0.46400505074079845,0.05916679528384143,0.2404031179999997,0.07159840288972932,0.6776133919999978,0.03673356736065802,291
244138713,329.655421048233,-49.5327632514873,0,True,3,1.57120225000381,0.0,99.0,21.041607,20.544988219999997,20.443959670547198,20.252493596,20.33,99.0,0.0266532,0.0224325,0.05367,0.0522095,0.158489,des,244138713,329.655421048233,-49.5327632514873,0,True,3,1.57120225000381,0.0,99.0,21.041607,20.544988219999997,20.443959670547198,20.252493596,20.33,99.0,0.0266532,0.0224325,0.05367,0.0522095,0.158489,-1,-1,329.04,-49.522,26.8605,1853.0,0.0,0.0,0.0,,7155,67663,23.97625,187.33458,-1,AAT,AAT,0.036534652,3.335641e-05,1,1,pgc67663_1_MG.mz,AAT,-1,True,329.6553927246441,-49.5327743468548,-1,23.52159835271868,nan,77.958393,99.00000358784372,0.49661878000000215,0.034836907533390506,0.10102854945279915,0.0581694589647351,0.191466074547197,0.07487523482600907,-0.07750640399999753,0.16686699796919102,0.29249462399999615,0.05682472126196485,329
244135825,329.355366165065,-49.4730649303556,0,True,3,1.82474120664597,0.0,99.0,20.0495645,19.52340517,19.240717134731202,19.066587106,18.9917,99.0,0.0132087,0.0130668,0.0289324,0.020966,0.0554367,des,244135825,329.355366165065,-49.4730649303556,0,True,3,1.82474120664597,0.0,99.0,20.0495645,19.52340517,19.240717134731202,19.066587106,18.9917,99.0,0.0132087,0.0130668,0.0289324,0.020966,0.0554367,-1,-1,329.04,-49.522,26.8605,1853.0,0.0,0.0,0.0,,7155,67663,12.635314,98.72458,-1,AAT,AAT,0.94947463,3.335641e-05,1,384,pgc67663_1_MG.mz,AAT,-1,True,329.3553920231136,-49.47307214460217,-1,22.82486123829543,nan,78.9504355,99.00000088116037,0.5261593299999987,0.01857985516439781,0.28268803526879793,0.031746260126194394,0.1741300287312022,0.03573033621112457,0.07488710599999848,0.05926888612830512,0.45681806400000013,0.024704542461660768,469
244955394,328.959637044431,-49.2945522201577,0,True,3,1.48953531932831,0.0,99.0,21.194961,20.64775706,20.3492229303488,20.124623107999998,20.374

Maybe because the err and the result are similar?

### Computing a target catalog with favor_highpri

In [211]:
tt = writecat['TargetName', 'TargetType']
tt['TargetName'].name = 'OBJID'
tt['idx'] = np.arange(len(tt))
jt = table.join(tt, tcat, join_type='left')
jt = jt[np.argsort(jt['idx'])]


subsampledcat2 = subsample_catalog(writecat, {1:100}, maxflux=5, maxguides=100, favor_highpri=jt['TARGETING_SCORE'])
cutpri = subsampledcat2[subsampledcat2['Priority']==1]
tcat['TARGETING_SCORE'][np.in1d(tcat['OBJID'], cutpri['TargetName'])]

Nflux: 5 Nguide: 92 NSky 78
Targets in each priority: {1: 100, 2: 76, 3: 22, 4: 50, 5: 126, 6: 92, 7: 39, 8: 4}
Total: 684


810
810
811
811
814
814
814
814
814
815
815
